In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "What do you know about Inception?",
        "answer": """
        I know these:
        Director: Christopher Nolan
        Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy
        Budget: $160 million
        Box Office Revenue: $836.8 million
        Genre: Science Fiction, Action, Thriller
        Synopsis: Dom Cobb, a skilled thief who steals information by infiltrating his targets' dreams, is offered a chance to have his criminal record erased. To do so, he must successfully plant an idea into a target's subconscious — a process known as inception. As Cobb and his team delve deeper into dream layers, the line between reality and illusion begins to blur.
        """,
    },
    {
        "question": "What do you know about Titanic?",
        "answer": """
        I know these:
        Director: James Cameron
        Main Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates
        Budget: $200 million
        Box Office Revenue: $2.25 billion
        Genre: Romance, Drama, Historical
        Synopsis: The epic love story of Jack Dawson and Rose DeWitt Bukater unfolds on the ill-fated maiden voyage of the RMS Titanic. Despite their different social classes, the two form a deep connection. However, their romance is cut short by the catastrophic sinking of the ship, turning their story into a timeless tale of love, sacrifice, and survival.
        """,
    },
    {
        "question": "What do you know about The Godfather?",
        "answer": """
        I know these:
        Director: Francis Ford Coppola
        Main Cast: Marlon Brando, Al Pacino, James Caan, Robert Duvall
        Budget: $6-7.2 million
        Box Office Revenue: $250-291 million
        Genre: Crime, Drama
        Synopsis: Set in 1940s New York, "The Godfather" follows the powerful and secretive Corleone crime family, led by patriarch Vito Corleone. When his youngest son Michael is reluctantly drawn into the family's operations, he transforms from a reluctant outsider to a ruthless leader, navigating a world of loyalty, betrayal, and violent retribution.
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {movie}?",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({"movie": "Avengers: Endgame"})

AI:
        I know these:
        Directors: Anthony Russo, Joe Russo
        Main Cast: Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, Jeremy Renner
        Budget: $356 million
        Box Office Revenue: $2.798 billion
        Genre: Superhero, Action, Adventure
        Synopsis: In the aftermath of the devastating events of "Avengers: Infinity War," the remaining Avengers must come together to undo the actions of the villainous Thanos and restore balance to the universe. With the fate of the world at stake, the Avengers embark on a final, epic battle that will determine the future of all existence.

AIMessageChunk(content='AI:\n        I know these:\n        Directors: Anthony Russo, Joe Russo\n        Main Cast: Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, Jeremy Renner\n        Budget: $356 million\n        Box Office Revenue: $2.798 billion\n        Genre: Superhero, Action, Adventure\n        Synopsis: In the aftermath of the devastating events of "Avengers: Infinity War," the remaining Avengers must come together to undo the actions of the villainous Thanos and restore balance to the universe. With the fate of the world at stake, the Avengers embark on a final, epic battle that will determine the future of all existence.')